In [1]:
from syft_rds.orchestra import setup_rds_stack
from syft_rds import RDS_NOTEBOOKS_PATH

In [2]:
stack = setup_rds_stack(log_level="INFO")

2025-03-07 10:31:43.892 | WARNING  | syft_rds.orchestra:setup_rds_stack:89 - Using shared data directory for both clients. Any file permission checks will be skipped as both clients have access to the same files.
2025-03-07 10:31:43.899 | INFO     | syft_rds.orchestra:setup_rds_stack:112 - Launching mock RDS stack in /var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_nq8uplfx...


In [3]:
do_client = stack.do_rds_client
ds_client = stack.ds_rds_client

# Create a dataset as DO

In [4]:
CWD = RDS_NOTEBOOKS_PATH / "quickstart"

In [5]:

private_dir = CWD / "data" / "dataset-1" / "private"
mock_dir = CWD / "data" / "dataset-1" / "mock"
markdown_path = CWD / "data" / "dataset-1" / "description.md"

private_dir.mkdir(parents=True, exist_ok=True)
mock_dir.mkdir(parents=True, exist_ok=True)


with open(private_dir / "data.csv", "w") as f:
    f.write("-1,-2,-3")

with open(mock_dir / "data.csv", "w") as f:
    f.write("1,2,3")

with open(markdown_path, "w") as f:
    f.write("some description")


In [6]:
data = do_client.dataset.create(
    name="dataset-1",  # MUST BE UNIQUE. Throw Exception if already exist.
    path=private_dir,  # MUST EXIST
    mock_path=mock_dir,
    summary="dummy data",
    description_path=markdown_path,
)

In [7]:
do_client.datasets

[Dataset
   uid: 30fb05f2-27a1-4be4-a841-5e7fb1b6123b
   created_at: 2025-03-07T09:31:44.033458Z
   updated_at: 2025-03-07T09:31:44.033463Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: dummy data
   readme: syft://data_owner@test.openmined.org/public/datasets/dataset-1/description.md
   tags: []]

# Use dataset as DS

In [8]:
ds_client.datasets

[Dataset
   uid: 30fb05f2-27a1-4be4-a841-5e7fb1b6123b
   created_at: 2025-03-07T09:31:44.033458Z
   updated_at: 2025-03-07T09:31:44.033463Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: dummy data
   readme: syft://data_owner@test.openmined.org/public/datasets/dataset-1/description.md
   tags: []]

In [9]:
ds_client.dataset.get("dataset-1").get_mock_path()


PosixPath('/private/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_nq8uplfx/shared_client_dir/datasites/data_owner@test.openmined.org/public/datasets/dataset-1')

In [10]:
ds_client.dataset.get("dataset-1").get_private_path()

PosixPath('/private/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_nq8uplfx/shared_client_dir/datasites/data_owner@test.openmined.org/private/datasets/dataset-1')

In [11]:
dataset = ds_client.dataset.get("dataset-1")
dataset.describe()

├── data.csv
└── description.md


True

In [12]:
import pathlib
from syft_rds.models.models import Dataset
from syft_rds.utils.render_code_template import render_data_access_template

In [13]:
my_file = CWD / "data" / "code1.py"

In [14]:
%%writefile {my_file}

import os

DATA_DIR = os.environ["DATA_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

print("Hello, world!")

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    f.write("ABC")


Overwriting /Users/koen/workspace/rds/notebooks/quickstart/data/code1.py


In [15]:
job = ds_client.jobs.submit(
    user_code_path=my_file,
    dataset_name = dataset.name
)
job


Job
  uid: cde5fda7-8de3-4ee2-89ee-cc3b1c731353
  created_at: 2025-03-07T09:31:44.301955Z
  updated_at: 2025-03-07T09:31:44.301958Z
  name: hybrid_stack_mqa7
  description: None
  user_code_id: 678b6cb1-104f-4a45-9f8b-a6ffef8cab37
  tags: []
  user_metadata: {}
  status: pending_code_review
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/cde5fda7-8de3-4ee2-89ee-cc3b1c731353
  dataset_name: dataset-1

In [16]:
ds_client

In [17]:
jobs = do_client.jobs.get_all()

In [18]:
job = jobs[-1]

In [19]:
job

Job
  uid: cde5fda7-8de3-4ee2-89ee-cc3b1c731353
  created_at: 2025-03-07T09:31:44.301955Z
  updated_at: 2025-03-07T09:31:44.301958Z
  name: hybrid_stack_mqa7
  description: None
  user_code_id: 678b6cb1-104f-4a45-9f8b-a6ffef8cab37
  tags: []
  user_metadata: {}
  status: pending_code_review
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/cde5fda7-8de3-4ee2-89ee-cc3b1c731353
  dataset_name: dataset-1

In [32]:
res_job = do_client.run_private(job)

2025-03-07 10:32:51.713 | WARNING  | syft_rds.client.rds_client:run_private:119 - Running job without docker is not secure


╭───────────────────────────── Job Configuration ──────────────────────────────╮
│ Starting job                                                                 │
│ Execution: python code1.py                                                   │
│ Dataset Dir.:  .../data_owner@test.openmined.org/private/datasets/dataset-1  │
│ Output Dir.:   .../tmp/syft-rds-jobs/cde5fda78de34ee289eecc3b1c731353/output │
│ Timeout:  60s                                                                │
╰──────────────────────────────────────────────────────────────────────────────╯

Running job...

Hello, world!

Job completed successfully!

In [38]:
do_client.jobs.share_results(res_job);

2025-03-07 10:33:56.582 | INFO     | syft_rds.client.rds_clients.jobs:share_results:94 - Shared results for job cde5fda7-8de3-4ee2-89ee-cc3b1c731353 at /private/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_nq8uplfx/shared_client_dir/datasites/data_owner@test.openmined.org/api_data/RDS/output/cde5fda7-8de3-4ee2-89ee-cc3b1c731353


In [34]:
from syft_rds.models.models import GetAllRequest

In [35]:
#job.add_to_queue()

In [40]:
job = ds_client.jobs.get_all()[-1]

In [43]:
job.output_url

'syft://data_owner@test.openmined.org/api_data/RDS/output/cde5fda7-8de3-4ee2-89ee-cc3b1c731353'